In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()

])

full_dataset = datasets.ImageFolder(root='/kaggle/input/rgb-arabic-alphabets-sign-language-dataset/RGB ArSL dataset', transform=transform)

In [ ]:
print(full_dataset.classes) 
print(full_dataset.class_to_idx)
print("Train examples:", len(train_dataset))    # should be > 0
print("Val   examples:", len(val_dataset))      # should be > 0


In [1]:
from torch.utils.data import Dataset
class HFImageDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]  # image is a torch.Tensor here
        # Convert tensor to PIL before processing (processor expects PIL or np.array)
        from torchvision.transforms.functional import to_pil_image
        image = to_pil_image(image)
        encoded = self.processor(images=image, return_tensors="pt")
        return {
            "pixel_values": encoded["pixel_values"].squeeze(0),  # remove batch dimension
            "labels": label
        }
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_raw, val_raw = random_split(full_dataset, [train_size, val_size])
train_dataset = HFImageDataset(train_raw, processor)
val_dataset   = HFImageDataset(val_raw, processor)

NameError: name 'full_dataset' is not defined

In [ ]:
from torch.utils.data import random_split

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])


In [ ]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)

In [1]:
from transformers import SiglipForImageClassification, AutoImageProcessor

model_name = "prithivMLmods/Hand-Gesture-19"
model = SiglipForImageClassification.from_pretrained(model_name,num_labels=31,ignore_mismatched_sizes =True)
processor = AutoImageProcessor.from_pretrained(model_name,use_fast = True)


config.json:   0%|          | 0.00/1.75k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/372M [00:00<?, ?B/s]

Some weights of SiglipForImageClassification were not initialized from the model checkpoint at prithivMLmods/Hand-Gesture-19 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([19]) in the checkpoint and torch.Size([31]) in the model instantiated
- classifier.weight: found shape torch.Size([19, 768]) in the checkpoint and torch.Size([31, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


preprocessor_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

In [1]:
import os
import cv2
import torch
from torch.utils.data import Dataset
from transformers import AutoImageProcessor

processor = AutoImageProcessor.from_pretrained("google/siglip-base-patch16-224",use_fast = True)

class ArSLDataset(Dataset):
    def __init__(self, root_dir, processor):
        self.samples = []
        self.labels_map = {}
        self.processor = processor

        label_id = 0
        for dirname, _, filenames in os.walk(root_dir):
            label = os.path.basename(dirname)
            if not filenames or label.startswith("."):
                continue  # skip hidden or empty folders
            if label not in self.labels_map:
                self.labels_map[label] = label_id
                label_id += 1
            for filename in filenames:
                filepath = os.path.join(dirname, filename)
                self.samples.append((filepath, self.labels_map[label]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        try:
            # Use cv2 to read the image (convert to RGB)
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        except:
            print(f"Failed to load image: {img_path}")
            image = 255 * np.ones((224, 224, 3), dtype=np.uint8)  # fallback: white image

        encoding = self.processor(images=image, return_tensors="pt")
        return {
            "pixel_values": encoding["pixel_values"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long)
        }

# Usage
root = "/kaggle/input/rgb-arabic-alphabets-sign-language-dataset/RGB ArSL dataset"
dataset = ArSLDataset(root, processor)

# Example split
from torch.utils.data import random_split


train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


2025-05-09 11:09:24.141669: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746788964.601478      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746788964.733999      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/368 [00:00<?, ?B/s]

In [2]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [ ]:
pip install --upgrade transformers


In [3]:
import transformers
from transformers import TrainingArguments, Trainer
from transformers import DefaultDataCollator
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}
    
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    eval_strategy ="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    report_to="none",  
    load_best_model_at_end=True,
    
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    processing_class=processor,
    compute_metrics=compute_metrics
)




NameError: name 'model' is not defined

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

SiglipForImageClassification(
  (vision_model): SiglipVisionTransformer(
    (embeddings): SiglipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), padding=valid)
      (position_embedding): Embedding(196, 768)
    )
    (encoder): SiglipEncoder(
      (layers): ModuleList(
        (0-11): 12 x SiglipEncoderLayer(
          (layer_norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (self_attn): SiglipAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): SiglipMLP(
            (activation_fn): PytorchGELUTanh()
            (fc1): Linear(in_features=768, out_feat

In [ ]:
print(transformers.__version__)

In [5]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# now all subsequent ImageFolder/PIL loads will ignore small truncation errors
from torchvision import datasets, transforms
trainer.train()


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.629500,0.333186,0.896947
2,0.125300,0.209412,0.944656
3,0.050500,0.105258,0.973919
4,0.024300,0.093064,0.977735
5,0.006800,0.051008,0.987277


Premature end of JPEG file
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
Premature end of JPEG file
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
Premature end of JPEG file
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
Premature end of JPEG file
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warning

TrainOutput(global_step=495, training_loss=0.16727933125062422, metrics={'train_runtime': 4550.7919, 'train_samples_per_second': 6.904, 'train_steps_per_second': 0.109, 'total_flos': 2.6322305261930496e+18, 'train_loss': 0.16727933125062422, 'epoch': 5.0})

In [ ]:
print("Trainer device:", training_args.device)


In [6]:
for param in model.vision_model.parameters():
    param.requires_grad = False
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.002700,0.051350,0.987277
2,0.002000,0.051646,0.987277
3,0.001700,0.051794,0.987277
4,0.001700,0.051826,0.987277
5,0.001600,0.051934,0.987277


Premature end of JPEG file
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
Premature end of JPEG file
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
Premature end of JPEG file
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
Premature end of JPEG file
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warning

TrainOutput(global_step=495, training_loss=0.001938977957975985, metrics={'train_runtime': 3967.7525, 'train_samples_per_second': 7.919, 'train_steps_per_second': 0.125, 'total_flos': 2.6322305261930496e+18, 'train_loss': 0.001938977957975985, 'epoch': 5.0})

In [7]:
metrics = trainer.evaluate()
print(metrics)

trainer.save_model("./finetuned-hand-signs-no-overfit")

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Premature end of JPEG file


{'eval_loss': 0.051350269466638565, 'eval_accuracy': 0.9872773536895675, 'eval_runtime': 156.3156, 'eval_samples_per_second': 10.057, 'eval_steps_per_second': 0.16, 'epoch': 5.0}


In [ ]:
""""from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

for epoch in range(10):
    resnet18.train()
    itera = 0
    for images, labels in train_loader:
        itera +=1
        images, labels = images.to(device), labels.to(device)
        
        outputs = resnet18(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()   
    print(f"epoch {epoch+1}, loss: {loss.item():.4f}")""""

In [ ]:
"""resnet18.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = resnet18(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation Accuracy: {100 * correct / total:.2f}%")
"""

In [1]:
import shutil

# Define source folder and output zip path (no need to add .zip to output name)
source_folder = '/kaggle/working/finetuned-hand-signs-no-overfit'
output_zip = '/kaggle/working/output_final_vit'  # This will create output.zip

# Create the zip archive
shutil.make_archive(output_zip, 'zip', source_folder)

'/kaggle/working/output_final_vit.zip'

In [ ]:
print("working")

In [4]:
from transformers import AutoImageProcessor, ResNetForImageClassification

processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")

res_model = ResNetForImageClassification.from_pretrained(
    "microsoft/resnet-50",
    num_labels=31,ignore_mismatched_sizes=True
)

preprocessor_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

Some weights of ResNetForImageClassification were not initialized from the model checkpoint at microsoft/resnet-50 and are newly initialized because the shapes did not match:
- classifier.1.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([31]) in the model instantiated
- classifier.1.weight: found shape torch.Size([1000, 2048]) in the checkpoint and torch.Size([31, 2048]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from transformers import TrainingArguments
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}
    
training_args = TrainingArguments(
    output_dir="./results_resnet",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    logging_dir="./logs",
    load_best_model_at_end=True,
)
from transformers import Trainer

trainer = Trainer(
    model=res_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    processing_class = processor,
    compute_metrics=compute_metrics
)


In [8]:
trainer.train()

<IPython.core.display.Javascript object>

KeyboardInterrupt: 